In [ ]:
import torch
from torch import nn
import math

In [2]:
x = torch.randn(32 * 64, 250, 64).cuda()
y = (torch.arange(0, 32*64) % 64).cuda()
loss_func = nn.CrossEntropyLoss().cuda()

In [3]:
query = nn.Linear(64, 64).cuda()
key = nn.Linear(64, 64).cuda()
value = nn.Linear(64, 64).cuda()

In [ ]:
for _ in range(9):
    q = query(x)
    k = key(x)
    v = value(x)

    attn = torch.matmul(q, k.transpose(-2, -1))
    attn = attn / math.sqrt(64)
    attn = nn.functional.softmax(attn, dim=-1)
    out = torch.matmul(attn, v) 
    x = out

logits = out.mean(1)
loss = loss_func(logits, y)
loss.backward()
loss

In [ ]:
for _ in range(9):
    out = []
    for idx in range(250):
        x2 = x[:, idx:idx+1, :]
        q = query(x2)
        k = key(x)
        v = value(x)
    
        attn = torch.matmul(q, k.transpose(-2, -1))
        attn = attn / math.sqrt(64)
        attn = nn.functional.softmax(attn, dim=-1)
        tmp = torch.matmul(attn, v) 
        out.append(tmp)
    
    out = torch.cat(out, dim=1)

logits = out.mean(1)
loss = loss_func(logits, y)
loss.backward()
loss